In [1]:
import docx
from docx import Document

In [7]:
import docx
import copy
import os
import xlrd

files_dir = "02_files"


def get_namecard_data(xls):
    """
    获取名片的数据
    :param xls:
    :return:
    """
    # 打开文件
    workbook = xlrd.open_workbook(xls)
    # 获取第一个工作表，从0开始
    sheet = workbook.sheet_by_index(0)
    # 名片信息列表（列表中的列表）
    namecard_infos_list = []
    # 从 0 开始
    field_list = sheet.row_values(0)
    for row_index in range(1, sheet.nrows):
        namecard_infos_list.append(sheet.row_values(row_index))
    # 返回了 2 个数据，1）名片字段列表 2）名片信息列表（列表中的列表）
    return field_list, namecard_infos_list


# Document 、Paragraph 和 Run 对象的
# Document 对象表示整个文档；
# Paragrapha 对象表示段落（在文档中，每一次回车会产生新段落）
# Run 对象表示相同样式的文本延续
# Document 对象包含一个 Paragrapha 对象的列表，Paragraph 对象包含一个 Run 对象的列表。

def replace_doc(document, namecard_values):
    """
    替换文档中的 [姓名] [职位] 为 指定的名片信息
    :param document: 包含 [姓名] [职位]...的文档对象
    :param namecard_values: 指定的名片信息（列表），索引的顺序 和 namecard_field_list 一致
    :return:
    """

    def replace_values(inline):
        """
        用于替换 inline 对象中的值，比如：[姓名] => 张三
        (**注意** 在 replace_doc 函数中 创建的函数，只能在 replace_doc函数 中使用)
        :param inline:
        :return:
        """
        # 循环遍历字段名（在循环中注释，字段名以“姓名”为例）
        for field_name in namecard_field_list:
            # field_name=姓名 ， field_var=[姓名]
            field_var = "[%s]" % field_name
            # 判断条件 inline.text 中是否包含 field_var（[姓名]） 字符串，有才进行替换，否则什么都不做
            if field_var in inline.text:
                # 获取字段的索引号
                field_index = namecard_field_list.index(field_name)
                # 获取“姓名”字段的文本内容，比如：text = 张三
                text = namecard_values[field_index]
                # 将 inline.text 中“[姓名]”替换为“张三”后，再赋值给 inline.text
                inline.text = inline.text.replace(field_var, text)

    # 循环遍历段落
    for paragraph in document.paragraphs:
        # 历获取段落中的文字
        text = paragraph.text
        # 判断段落中的文字中是否包含 [ 并且也包含 ] ,如果为 True，则认为需要替换
        if ("[" in text) and (']' in text):
            # Paragraph 对象包含一个 Run 对象的列表
            # Run 对象表示相同样式的文本延续
            inlines = paragraph.runs
            # 相同样式的文本延续
            for inline in inlines:
                # 用于替换 inline 对象中的值，比如：[姓名] => 张三
                replace_values(inline)

    return document


# 存放名片的字段（变量，需要修改的）
namecard_field_list, namecard_infos_list = get_namecard_data(f"./{files_dir}/人员名片信息.xlsx")
print(namecard_field_list)
# ['姓名', '职位', '部门', '电话', '手机', '邮箱']
print(namecard_infos_list)
# [['张三', '开发工程师', '技术部', '87646000', '13399999999', 'zhangsan@abcde.com.cn'],
# ['李四', '业务经理', '业务部', '87646001', '15600000000', 'lisi@abcde.com.cn']]

name_card_doc_tpl = f"./{files_dir}/Word_名片模版.docx"
# 获取文档对象
name_card_tpl = docx.Document(name_card_doc_tpl)

# 循环遍历名片信息列表（多个人员的名片数据）
for namecard_info in namecard_infos_list:
    # 复制一份模板，防止替换后之前的模版被替换了
    name_card_doc = copy.deepcopy(name_card_tpl)
    # 开始替换
    replace_doc(name_card_doc, namecard_info)
    # 去人员姓名
    staff_name = namecard_info[0]
    name_card_doc.save(f'./{files_dir}/{staff_name}名片.docx')


['姓名', '职位', '部门', '电话', '手机', '邮箱']
[['张三', '开发工程师', '技术部', '87646000', '13399999999', 'zhangsan@abcde.com.cn'], ['李四', '业务经理', '业务部', '87646001', '15600000000', 'lisi@abcde.com.cn']]
